In [1]:
import os
import glob
import numpy as np
import SimpleITK as sitk
import random
from keras.applications.resnet50 import ResNet50
from keras.optimizers import SGD
import cv2

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
path = "/data/gyungchan2110/Classification/DataSet_Cho_20180329_151600"

train_files = glob.glob(os.path.join(path,'train', '*', '*.png'))
test_files = glob.glob(os.path.join(path, 'validation', '*', '*.png'))

model = ResNet50(weights=None, include_top=True, input_shape=(512, 512, 3),classes = 2)
optimizer = SGD(lr=1e-5, decay=1e-5, momentum=0.9, nesterov=True) # decay = 1e-4
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

/home/gyungchan2110/gyungchan2110_python3/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


In [ ]:
count = 0
X_train = []
y_train = []

train_mean = 170.

random.shuffle(train_files)
random.shuffle(test_files)
for _ in range(10):
    losses = []
    for i in train_files:
        img = sitk.GetArrayFromImage(sitk.ReadImage(i)).astype('float32')
        img = cv2.resize(img, (512, 512))
        img -= train_mean
        
        #print(np.amax(img.flatten()), np.amin(img.flatten()), img.mean())
        
        if i.split('/')[-2].startswith('N'):
            y = np.array([1, 0])
        else:
            y = np.array([0, 1])

        X_train.append(img)
        y_train.append(y)
        count += 1
        if count % 4 == 0:
            X_train, y_train = np.array(X_train), np.array(y_train)
            loss = model.train_on_batch(X_train, y_train)
            
            count = 0
            X_train, y_train = [], []
            losses.append(loss)
        
    losses = np.asarray(losses)
    print("train loss", losses.mean())
    losses = []
    for i in test_files:
        img = sitk.GetArrayFromImage(sitk.ReadImage(i)).astype('float32')
        img = cv2.resize(img, (512, 512))
        img -= train_mean
        
        #print(np.amax(img.flatten()), np.amin(img.flatten()), img.mean())
        
        if i.split('/')[-2].startswith('N'):
            y = np.array([1, 0])
        else:
            y = np.array([0, 1])

        X_train.append(img)
        y_train.append(y)
        count += 1
        if count % 4 == 0:
            X_train, y_train = np.array(X_train), np.array(y_train)
            loss = model.test_on_batch(X_train, y_train)
            
            count = 0
            X_train, y_train = [], []
            losses.append(loss)
    losses = np.asarray(losses)
    print("vlid loss", losses.mean())
    losses = []
print('done')

train loss 0.63711435
vlid loss nan


/home/gyungchan2110/gyungchan2110_python3/lib/python3.4/site-packages/ipykernel_launcher.py:60: RuntimeWarning: Mean of empty slice.
/home/gyungchan2110/gyungchan2110_python3/lib/python3.4/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


train loss 0.62252575
vlid loss nan


In [5]:
valid_files = glob.glob(os.path.join(path, 'validation', '*', '*.png'))

random.shuffle(valid_files)
for i in valid_files[:10]:
    img = sitk.GetArrayFromImage(sitk.ReadImage(i)).astype('float32')
    img = cv2.resize(img, (224, 224))
    img -= train_mean
    
    if i.split('/')[-2].startswith('N'):
        y = np.array([1, 0])
    else:
        y = np.array([0, 1])
    
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    
    print(model.predict(img))
    print(y)

[[0.17527276 0.8247272 ]]
[1 0]
[[0.17429118 0.8257088 ]]
[1 0]
[[0.20733096 0.79266906]]
[0 1]
[[0.1866148 0.8133852]]
[1 0]
[[0.35048825 0.64951175]]
[1 0]
[[0.3597862 0.6402137]]
[0 1]
[[0.19474931 0.8052507 ]]
[0 1]
[[0.1846434 0.8153566]]
[1 0]
[[0.23302928 0.7669707 ]]
[1 0]
[[0.19785416 0.80214584]]
[1 0]


In [39]:
valid_files = glob.glob(os.path.join(path, 'validation', '*', '*.png'))

random.shuffle(valid_files)
for i in valid_files[:10]:
    img = sitk.GetArrayFromImage(sitk.ReadImage(i)).astype('float32')
    img = cv2.resize(img, (224, 224))
    img -= train_mean
    
    if i.split('/')[-2].startswith('N'):
        y = np.array([1, 0])
    else:
        y = np.array([0, 1])
    
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    
    print(model.predict(img))
    print(y)

[[0.2674347  0.73256534]]
[1 0]
[[0.261067   0.73893297]]
[1 0]
[[0.2628673 0.7371327]]
[0 1]
[[0.26938254 0.73061746]]
[0 1]
[[0.2636571  0.73634297]]
[1 0]
[[0.26654667 0.73345333]]
[0 1]
[[0.2803348 0.7196652]]
[0 1]
[[0.25668827 0.7433117 ]]
[1 0]
[[0.27178228 0.7282177 ]]
[1 0]
[[0.25810298 0.741897  ]]
[1 0]


In [32]:
test_files = glob.glob(os.path.join(path, 'test', '*', '*.png'))

for i in test_files[:10]:
    img = sitk.GetArrayFromImage(sitk.ReadImage(i)).astype('float32')
    img = cv2.resize(img, (224, 224))
    img -= train_mean
    
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    
    print(model.predict(img))

[[0.20486844 0.79513156]]
[[0.27822703 0.72177297]]
[[0.27167597 0.728324  ]]
[[0.19695605 0.8030439 ]]
[[0.21497364 0.7850264 ]]
[[0.20889738 0.7911026 ]]
[[0.21477401 0.7852259 ]]
[[0.45785716 0.5421428 ]]
[[0.35081783 0.64918214]]
[[0.16244237 0.8375577 ]]


In [1]:
from Util import prepare_Task, TaskID_Generator
import os 
from Config import Config
import ClassificationTask 

GPU_Num, BaseDataPath = prepare_Task("Classification")
TaskID = TaskID_Generator() 
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_Num
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Classes = []
configs = []

Classes.append("Normal")
Classes.append("Abnormal")



config = Config("Cardiomegaly", (224,224,1), Classes, "DataSet_Cho_20180329_151600")
config.MODEL_TYPE = "RESNET50"
config.BATCH_SIZE = 32
config.OPTIMIZER = "SGD"
config.LEARNING_RATE = 1e-4
config.DECAY = 1e-5
configs.append(config)

# config = Config("Cardiomegaly", (224,224,3), Classes, "Generated_Data_20180327_151800_2Classes_Original")
# config.MODEL_TYPE = "INCEPTIONV3"
# configs.append(config)

for i in range(len(configs)):
    result , model, filename, mean = ClassificationTask.Train(TaskID, BaseDataPath, configs[i])
    if(result):
        ClassificationTask.Test(TaskID, BaseDataPath, configs[i], filename, mean)
    

/home/gyungchan2110/gyungchan2110_python3/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


/data/gyungchan2110/Classification/DataSet_Cho_20180329_151600/train/Normal
/data/gyungchan2110/Classification/DataSet_Cho_20180329_151600/train/Abnormal
/data/gyungchan2110/Classification/DataSet_Cho_20180329_151600/validation/Normal
/data/gyungchan2110/Classification/DataSet_Cho_20180329_151600/validation/Abnormal
Epoch 1/200
117/117 [==============================] - 51s 434ms/step - loss: 0.6128 - acc: 0.6744 - val_loss: 4.7876 - val_acc: 0.4993
Epoch 2/200
117/117 [==============================] - 49s 421ms/step - loss: 0.5250 - acc: 0.7288 - val_loss: 1.6167 - val_acc: 0.5207
Epoch 3/200
117/117 [==============================] - 50s 426ms/step - loss: 0.4109 - acc: 0.8293 - val_loss: 0.6662 - val_acc: 0.6507
Epoch 4/200
117/117 [==============================] - 50s 426ms/step - loss: 0.2883 - acc: 0.8965 - val_loss: 0.4399 - val_acc: 0.7740
Epoch 5/200
117/117 [==============================] - 50s 427ms/step - loss: 0.1835 - acc: 0.9434 - val_loss: 0.3217 - val_acc: 0.8722
Ep

KeyboardInterrupt: 

In [1]:
from Util import prepare_Task, TaskID_Generator
import os 
from Config import Config
import ClassificationTask 

GPU_Num, BaseDataPath = prepare_Task("Classification")
TaskID = TaskID_Generator() 
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_Num
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Classes = []
configs = []

Classes.append("Normal")
Classes.append("Abnormal")



config = Config("Cardiomegaly", (224,224,1), Classes, "DataSet_Cho_20180329_151600")
config.MODEL_TYPE = "RESNET50"
config.BATCH_SIZE = 16
config.OPTIMIZER = "SGD"

configs.append(config)

filename = "RESNET50_20180403_202701.hdf5_010.hdf5"

for i in range(len(configs)):
    ClassificationTask.Test(TaskID, BaseDataPath, configs[i], filename)

/home/gyungchan2110/gyungchan2110_python3/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


/data/gyungchan2110/Classification/DataSet_Cho_20180329_151600/test/Normal
/data/gyungchan2110/Classification/DataSet_Cho_20180329_151600/test/Abnormal
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0],